**Important!** Due to some libraries not being updated, this notebook requires Python 3.11 and won't work with any newer versions. Please make sure your environment meets this requirement.

In [ ]:
!pip uninstall torch
!pip uninstall torchaudio
!rm -rf /usr/local/lib/python*/dist-packages/~orch*
!pip install torch==2.2.0 torchaudio==2.2.0

In [ ]:
!pip uninstall transformers
!pip install transformers==4.33.0

In [ ]:
!pip install TTS

You can skip the cell below if you don't want to save the results to Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import torch
from TTS.api import TTS
import glob
import numpy as np
import random


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
command = "Wake up, Typist!"

In [ ]:
tts = TTS(model_name="tts_models/multilingual/multi-dataset/xtts_v1.1", progress_bar=True).to(device)

In [ ]:
emotions = ["Happy", "Sad", "Angry", "Dull"]

In [ ]:
for num in range(292,377):
  files = glob.glob("/content/drive/MyDrive/vctk/p"+str(num)+"/*.wav") # replace with path to your samples
  if len(files) < 20:
    continue
  x = 0
  for em in emotions:
    tts.tts_to_file(
        text=command,
        language="en",
        file_path="/content/drive/MyDrive/wakeupset/sample"+str(num)+str(x)+".wav", # replace with path to results
        emotion=em,
        temperature=random.randint(7,10)/10.0,
        speaker_wav=files[:20]
    )
    x+=1